In [2]:
import pandas as pd
import numpy as np

* since the mean r2oos of other methods including elastic net, glm, pls, neural network are too small, we only did a dm test on gbrt and rf.

# recreate the train/validation/test split

In [3]:
# import pyarrow.feather as feather
import os
from datetime import datetime
from tqdm import tqdm


import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from collections import Counter

## read the feather files

In [4]:
import os
directory_path = os.getcwd()
print("My current directory is : " + directory_path)
folder_name = os.path.basename(directory_path)
print("My directory name is : " + folder_name)

My current directory is : /home/spectre/Desktop/summer_project/Summer-project-ML
My directory name is : Summer-project-ML


In [5]:
directory_path = os.getcwd()
data_path = os.path.join(directory_path, 'data')  # the path to data folder

In [6]:
raw_df = pd.read_feather(os.path.join(data_path, 'chars60_raw_imputed.feather'))

## check missing values

In [7]:
raw_tf = raw_df.isnull().any()
raw_tf.where(raw_tf == True).dropna().index.to_list()


['adm',
 'bm_ia',
 'mom12m',
 'mom36m',
 'mom60m',
 'mom6m',
 'seas1a',
 'abr',
 'sue',
 'cinvest',
 'nincr',
 'pscore',
 'acc',
 'bm',
 'agr',
 'alm',
 'ato',
 'cashdebt',
 'chcsho',
 'chpm',
 'chtx',
 'gma',
 'grltnoa',
 'lgr',
 'ni',
 'noa',
 'op',
 'pctacc',
 'rna',
 'roa',
 'roe',
 'rsup',
 'sgr']

In [8]:
raw_df['year'] = raw_df['date'].dt.year
raw_df = raw_df[raw_df['year'] >= 1972]
raw_df = raw_df.drop(['year'], axis=1)

In [9]:
raw_tf = raw_df.isnull().any()
raw_tf.where(raw_tf == True).dropna().index.to_list()

[]

## read the macro data files

In [10]:
macro_df = pd.read_csv(os.path.join(data_path, 'Macro.csv'))
macro_df = macro_df[macro_df['yyyymm']>=197201]

In [11]:
raw_df['yyyymm'] = raw_df['date'].dt.strftime("%Y%m").apply(int)
total_df = pd.merge(raw_df, macro_df, how='inner', on='yyyymm')
total_df.drop(columns="yyyymm", inplace=True)


In [12]:
raw_tf = total_df.isnull().any()
raw_tf.where(raw_tf == True).dropna().index.to_list()

[]

In [13]:
del(raw_tf)

In [14]:
# make a list of unique permno
permno_list = total_df['permno'].unique().tolist()

In [15]:
# need to turn some variables to dummy variables
def get_dummy_col(target_df, col_names):
    """turn the cols into multiple cols containing dummy variables, then drop the original columns

    Args:
        target_df (pandas dataframe): target dataframe
        col_names (list): list of columns that need to be turned into dummy variables
    """
    for col in col_names:        
        tmp_dummies = pd.get_dummies(target_df[col]).rename(columns=lambda x: f'{col}_' + str(x))
        target_df = pd.concat([target_df, tmp_dummies], axis=1)
        target_df.drop(col, inplace=True, axis = 1)
    return target_df 
dummy_col_list = ['ffi49']
total_df = get_dummy_col(total_df, dummy_col_list)

In [16]:
# set date as the index
total_df.set_index('date', inplace=True)

In [17]:
features_list = total_df.columns.to_list()
list_to_remove = ['gvkey', 'permno', 'ret', 'sic', 'exchcd', 'shrcd']
for element in list_to_remove:
    if element in features_list:
        features_list.remove(element)
        


In [18]:
original_features_list = features_list.copy()

In [19]:
total_df = total_df.sort_index()

# create interaction features(other feature * macro feature)

In [20]:
table_for_interaction = total_df[features_list[:69]]
single_features = features_list[:61]
macro_features = features_list[61:69]

interaction_table = pd.DataFrame(data = None, index = table_for_interaction.index)

from IPython.utils import io
with io.capture_output() as captured: # to ignore warnings
    for i in single_features:
        for j in macro_features:
            interaction_table[i + '+' + j] = table_for_interaction[i] * table_for_interaction[j]

In [21]:
each_month_permno_num = pd.DataFrame(Counter(total_df.reset_index().date).items(), columns=['date', 'number_of_permno'])
# each_month_permno_num.plot(x='date', y='number_of_permno')

In [22]:
new_total_df = pd.concat([total_df, interaction_table], axis=1)

In [23]:
new_features_list = new_total_df.columns
remove_list = {'gvkey', 'permno', 'ret','sic','exchcd','shrcd','b/m', 'tbl', 'ntis', 'svar', 'd/p', 'e/p', 'dfy', 'tms'}
new_features_list = [ele for ele in new_features_list if ele not in remove_list]

In [24]:
X_total = new_total_df[new_features_list]
y_total = new_total_df['ret']
permno_total = new_total_df['permno']

In [25]:
def train_val_test_preprocess(X_total, y_total, permno_total, training_len, val_len, testing_len):
    """split our data into training set, validation set, and testing set

    Args:
        X_total (dataframe)
        y_total (dataframe)
        training_len (int): length of years of training set
        val_len (int)
        testing_len (int)
    return: dicts:X_train, y_train, X_val, y_val, X_test, y_test
            and a permno series for testing set, which would be part of the result
    """
    X_train_dict = {}
    y_train_dict = {}
    X_val_dict = {}
    y_val_dict = {}
    X_test_dict = {}
    y_test_dict = {}
    permno_test_dict = {}
    first_train_end_year = y_total.index.year[0] + training_len - 1
    total_len = y_total.index.year[-1] - y_total.index.year[0] + 1
    last_train_end_year = first_train_end_year + total_len -(training_len+val_len+testing_len)
    train_end_years = np.arange(first_train_end_year, last_train_end_year+1)
    print('each end year of training set is:', train_end_years)
    for y in tqdm(train_end_years, desc='spliting data', ascii='>>'):
        train_start_year = y - training_len + 1
        train_end_year = y
        val_start_year = y + 1
        val_end_year = val_start_year+val_len-1
        test_start_year= val_end_year+1
        test_end_year = test_start_year+testing_len-1

        print('train start year is', train_start_year)
        print('train end year is', train_end_year)

        print('val start year is', val_start_year)
        print('val end year is', val_end_year)

        print('test start year is', test_start_year)
        print('test end year is', test_end_year)

        tmp_X_train = X_total[(X_total.index.year>=train_start_year) & (X_total.index.year<=train_end_year)]
        tmp_y_train = y_total[(y_total.index.year>=train_start_year) & (y_total.index.year<=train_end_year)]
        tmp_X_val = X_total[(X_total.index.year>=val_start_year) & (X_total.index.year<=val_end_year)]
        tmp_y_val = y_total[(y_total.index.year>=val_start_year) & (y_total.index.year<=val_end_year)]
        tmp_X_test = X_total[(X_total.index.year>=test_start_year) & (X_total.index.year<=test_end_year)]
        tmp_y_test = y_total[(y_total.index.year>=test_start_year) & (y_total.index.year<=test_end_year)]
        tmp_permno_test = permno_total[(permno_total.index.year>=test_start_year) & (permno_total.index.year<=test_end_year)]

        X_train_dict[y] = tmp_X_train
        y_train_dict[y] = tmp_y_train
        X_val_dict[y] = tmp_X_val
        y_val_dict[y] = tmp_y_val
        X_test_dict[y] = tmp_X_test
        y_test_dict[y] = tmp_y_test
        permno_test_dict[y] = tmp_permno_test
    return train_end_years, X_train_dict, y_train_dict, X_val_dict, y_val_dict, X_test_dict, y_test_dict, permno_test_dict
        

In [26]:
train_end_years, X_train_dict, y_train_dict, X_val_dict, y_val_dict, X_test_dict, y_test_dict, permno_test_dict = train_val_test_preprocess(X_total, y_total, permno_total, 14, 6, 26)


each end year of training set is: [1985 1986 1987 1988]


spliting data:   0%|>>>>>>>>>>| 0/4 [00:00<?, ?it/s]

train start year is 1972
train end year is 1985
val start year is 1986
val end year is 1991
test start year is 1992
test end year is 2017


spliting data:  25%|>>>>>>>>>>| 1/4 [00:00<00:02,  1.34it/s]

train start year is 1973
train end year is 1986
val start year is 1987
val end year is 1992
test start year is 1993
test end year is 2018


spliting data:  50%|>>>>>>>>>>| 2/4 [00:01<00:01,  1.33it/s]

train start year is 1974
train end year is 1987
val start year is 1988
val end year is 1993
test start year is 1994
test end year is 2019


spliting data:  75%|>>>>>>>>>>| 3/4 [00:02<00:00,  1.33it/s]

train start year is 1975
train end year is 1988
val start year is 1989
val end year is 1994
test start year is 1995
test end year is 2020


spliting data: 100%|>>>>>>>>>>| 4/4 [00:03<00:00,  1.33it/s]


----------------------------------------------------------

# preprocessing for dm_test
make the data more readable

In [27]:
for y in train_end_years:
    tmp_y_test = y_test_dict[y]
    print(tmp_y_test.index)

DatetimeIndex(['1992-01-31', '1992-01-31', '1992-01-31', '1992-01-31',
               '1992-01-31', '1992-01-31', '1992-01-31', '1992-01-31',
               '1992-01-31', '1992-01-31',
               ...
               '2017-12-31', '2017-12-31', '2017-12-31', '2017-12-31',
               '2017-12-31', '2017-12-31', '2017-12-31', '2017-12-31',
               '2017-12-31', '2017-12-31'],
              dtype='datetime64[ns]', name='date', length=238048, freq=None)
DatetimeIndex(['1993-01-31', '1993-01-31', '1993-01-31', '1993-01-31',
               '1993-01-31', '1993-01-31', '1993-01-31', '1993-01-31',
               '1993-01-31', '1993-01-31',
               ...
               '2018-12-31', '2018-12-31', '2018-12-31', '2018-12-31',
               '2018-12-31', '2018-12-31', '2018-12-31', '2018-12-31',
               '2018-12-31', '2018-12-31'],
              dtype='datetime64[ns]', name='date', length=235983, freq=None)
DatetimeIndex(['1994-01-31', '1994-01-31', '1994-01-31', '1994-01-

In [28]:
gbrt_result_df = pd.read_pickle('result_data/gbrt_testing26_basic_result.pkl')
rf_result_df = pd.read_pickle('result_data/rf_testing26_interaction_result_v2.pkl')


In [29]:
gbrt_result_df.columns

Index(['year', 'permno', 'y_pred', 'y_real', 'params', 'pred_R2_OOS', 'CV_R2',
       'CV_MSE'],
      dtype='object')

In [30]:
gbrt_result_df['year'].unique()

array([1985, 1986, 1987, 1988], dtype=object)

In [31]:
# tmp_y_real_series.index.duplicated().any()

In [32]:
def make_df_list(train_end_years, result_df):
    y_pred_df_list = []
    y_real_df_list = []
    for y in train_end_years:
        tmp_permno = pd.Series(result_df.loc[result_df['year'] == y, 'permno'].tolist()[0])
        tmp_y_pred_series = pd.Series(result_df.loc[result_df['year'] == y, 'y_pred'].to_list()[0])
        # print(len(tmp_y_pred_series))
        # tmp_y_real_series already has index
        tmp_y_real_series = pd.Series(result_df.loc[result_df['year'] == y, 'y_real'].to_list()[0])
        #add index to tmp_y_pred_series
        tmp_y_pred_series.index = tmp_y_real_series.index

        
        tmp_pred_df = pd.concat([tmp_permno, tmp_y_pred_series], axis=1)
        tmp_real_df = pd.concat([tmp_permno, tmp_y_real_series], axis=1)
        #reset_index
        tmp_pred_df.reset_index(drop=False, inplace=True)
        tmp_real_df.reset_index(drop=False, inplace=True)
        #rename the ret column
        tmp_pred_df.rename(columns = {0:f'ret_{y}'}, inplace = True)
        tmp_real_df.rename(columns = {'ret':f'ret_{y}'}, inplace = True)

        # print(len(tmp_y_real_series))
        y_pred_df_list.append(tmp_pred_df)
        y_real_df_list.append(tmp_real_df)
    return y_pred_df_list, y_real_df_list

In [33]:
gbrt_pred_df_list, gbrt_real_df_list = make_df_list(train_end_years, gbrt_result_df)
rf_pred_df_list, rf_real_df_list = make_df_list(train_end_years, rf_result_df)

In [34]:
gbrt_real_df_list[0].columns

Index(['date', 'permno', 'ret_1985'], dtype='object')

------------------------------------
# preprocessing for making portfolio 

In [35]:
# random forest
# merge all the dataframe together
merged_pred_df = pd.merge(rf_pred_df_list[0], rf_pred_df_list[1], on=['date', 'permno'], how='outer')
merged_pred_df = pd.merge(merged_pred_df, rf_pred_df_list[2], on=['date', 'permno'], how='outer')
merged_pred_df = pd.merge(merged_pred_df, rf_pred_df_list[3], on=['date', 'permno'], how='outer')

merged_real_df = pd.merge(rf_real_df_list[0], rf_real_df_list[1], on=['date', 'permno'], how='outer')
merged_real_df = pd.merge(merged_real_df, rf_real_df_list[2], on=['date', 'permno'], how='outer')
merged_real_df = pd.merge(merged_real_df, rf_real_df_list[3], on=['date', 'permno'], how='outer')


In [36]:
merged_pred_df

,date,permno,ret_1985,ret_1986,ret_1987,ret_1988
0,1992-01-31,18921,0.01192,NaN,NaN,NaN
1,1992-01-31,24942,0.01192,NaN,NaN,NaN
2,1992-01-31,46228,0.01192,NaN,NaN,NaN
3,1992-01-31,14154,0.01192,NaN,NaN,NaN
4,1992-01-31,91380,0.01192,NaN,NaN,NaN
...,...,...,...,...,...,...
260338,2020-12-31,60986,NaN,NaN,NaN,0.014906
260339,2020-12-31,21207,NaN,NaN,NaN,0.015153
260340,2020-12-31,57665,NaN,NaN,NaN,0.015153
260341,2020-12-31,57817,NaN,NaN,NaN,0.014857


In [37]:
merged_real_df.iloc[:, 2].to_list()

[-0.04318181797862053,
 0.02857142873108387,
 -0.1538461595773697,
 -0.03750000149011612,
 -0.03396226465702057,
 -0.09866666793823242,
 0.11340206116437912,
 -0.04651162773370743,
 -0.034302324056625366,
 0.11999999731779099,
 0.008093385025858879,
 0.00913241971284151,
 0.07758620381355286,
 0.2874999940395355,
 0.005240174941718578,
 0.32307693362236023,
 0.0,
 -0.12091503292322159,
 0.08645614236593246,
 0.0,
 0.15060728788375854,
 -0.05543237179517746,
 -0.05797101557254791,
 -0.08771929889917374,
 -0.09705093502998352,
 -0.04828660562634468,
 -0.015625,
 0.24626865983009338,
 -0.11279461532831192,
 0.03762376308441162,
 0.04444444552063942,
 0.07042253762483597,
 0.03921568766236305,
 -0.03999999910593033,
 0.023904383182525635,
 -0.08955223858356476,
 0.20000000298023224,
 0.11051930487155914,
 -0.01075268816202879,
 -0.02777777798473835,
 -0.02857142873108387,
 -0.06808510422706604,
 0.06666667014360428,
 0.18617020547389984,
 -0.1285475790500641,
 0.16552846133708954,
 0.06790

In [50]:
merged_pred_df.iloc[:, 5]

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
260338    0.014906
260339    0.015153
260340    0.015153
260341    0.014857
260342    0.014906
Name: ret_1988, Length: 260343, dtype: float64

In [54]:
merged_pred_df['ret_ave'] = merged_pred_df[['ret_1985', 'ret_1986', 'ret_1987', 'ret_1988']].mean(axis=1)
merged_real_df['ret_ave'] = merged_real_df[['ret_1985', 'ret_1986', 'ret_1987', 'ret_1988']].mean(axis=1)

In [55]:
merged_pred_df

,date,permno,ret_1985,ret_1986,ret_1987,ret_1988,ret_ave
0,1992-01-31,18921,0.01192,NaN,NaN,NaN,0.011920
1,1992-01-31,24942,0.01192,NaN,NaN,NaN,0.011920
2,1992-01-31,46228,0.01192,NaN,NaN,NaN,0.011920
3,1992-01-31,14154,0.01192,NaN,NaN,NaN,0.011920
4,1992-01-31,91380,0.01192,NaN,NaN,NaN,0.011920
...,...,...,...,...,...,...,...
260338,2020-12-31,60986,NaN,NaN,NaN,0.014906,0.014906
260339,2020-12-31,21207,NaN,NaN,NaN,0.015153,0.015153
260340,2020-12-31,57665,NaN,NaN,NaN,0.015153,0.015153
260341,2020-12-31,57817,NaN,NaN,NaN,0.014857,0.014857
